# **数据聚合与分组运算**
---
Group By
> 以一种自然的方式对数据集进行切片、切块、摘要等操作。  

在本章中，你将会学到：  

> 一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象。  
> 分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。  
> 组内转换或其他运算，如规格化、线性回归、排名或选取子集等。  
> 透视表或交叉表。  
> 分位数分析以及其它统计分组分析。  

笔记：对时间序列数据的聚合（groupby的特殊用法之一）也称作重采样（resampling），本书将在第11章中单独对其进行讲解。

## **Group By机制**

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                      'key2' : ['one', 'two', 'one', 'two', 'one'],
                      'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.212076,0.132477
1,a,two,-0.625259,0.173297
2,b,one,-0.998081,1.112946
3,b,two,1.431514,-1.256455
4,a,one,-0.179711,-0.230544


In [3]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.516182
      two    -0.625259
b     one    -0.998081
      two     1.431514
Name: data1, dtype: float64

In [4]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.516182 -0.049034
     two  -0.625259  0.173297
b    one  -0.998081  1.112946
     two   1.431514 -1.256455

### **对分组迭代**

In [5]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
        print((k1, k2))
        print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.212076  0.132477
4    a  one -0.179711 -0.230544
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.625259  0.173297
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.998081  1.112946
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.431514 -1.256455


> **可以将其转换为字典**

In [6]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.212076  0.132477
 1    a  two -0.625259  0.173297
 4    a  one -0.179711 -0.230544,
 'b':   key1 key2     data1     data2
 2    b  one -0.998081  1.112946
 3    b  two  1.431514 -1.256455}

In [7]:
grouped = df.groupby(df.dtypes, axis=1) # 也可以在列上分组

### **选取一列或列的子集进行计算**

In [8]:
df

,key1,key2,data1,data2
0,a,one,1.212076,0.132477
1,a,two,-0.625259,0.173297
2,b,one,-0.998081,1.112946
3,b,two,1.431514,-1.256455
4,a,one,-0.179711,-0.230544


In [9]:
df.groupby('key1')['data1'].head()
df.groupby('key1')[['data2']].head()

,data2
0,0.132477
1,0.173297
2,1.112946
3,-1.256455
4,-0.230544


In [10]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [11]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.049034
     two   0.173297
b    one   1.112946
     two  -1.256455

### **通过字典或Series进行分组**

In [12]:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.954614,0.428463,0.167765,-1.842625,0.066466
Steve,-1.327827,-0.089169,0.175448,2.669560,-1.002686
Wes,-0.102542,NaN,NaN,-2.005417,-0.312159
Jim,0.494672,-0.588247,-0.501183,0.136597,-0.917089
Travis,1.877813,-0.207497,-0.881921,-2.189427,-0.485967


In [13]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.674859,-0.459685
Steve,2.845007,-2.419682
Wes,-2.005417,-0.414701
Jim,-0.364586,-1.010664
Travis,-3.071348,1.184348


### **通过内置函数进行分组**

In [14]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.954614  0.428463  0.167765 -2.005417 -0.312159
  two  0.494672 -0.588247 -0.501183  0.136597 -0.917089
5 one -1.327827 -0.089169  0.175448  2.669560 -1.002686
6 two  1.877813 -0.207497 -0.881921 -2.189427 -0.485967

### **根据索引级别分组**

In [15]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                  names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.671805  0.421996 -0.054884  0.045830  1.141247
1     -0.008151  0.627276  0.490441 -0.888817  1.224454
2      0.797802  1.379986 -0.029597  0.199601 -0.424293
3      1.397739  0.108072 -0.185497 -0.058749  0.545147

In [16]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## **数据聚合**

In [17]:
df

,key1,key2,data1,data2
0,a,one,1.212076,0.132477
1,a,two,-0.625259,0.173297
2,b,one,-0.998081,1.112946
3,b,two,1.431514,-1.256455
4,a,one,-0.179711,-0.230544


In [18]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.933718
b    1.188555
Name: data1, dtype: float64

## **使用自定义函数分组**

In [19]:
def peak_to_peak(arr): # 定义自己的函数
          return arr.max() - arr.min()
grouped.agg(peak_to_peak) # 传入自己的函数

/var/folders/g3/9nr_2jgn5gg0yglwn47g6sfm0000gn/T/ipykernel_8909/3670497898.py:3: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak) # 传入自己的函数


,data1,data2
key1,,
a,1.837335,0.403841
b,2.429595,2.369401


In [20]:
grouped.agg(lambda x:x.max() - x.min()) # 使用匿名函数

/var/folders/g3/9nr_2jgn5gg0yglwn47g6sfm0000gn/T/ipykernel_8909/2587939441.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(lambda x:x.max() - x.min()) # 使用匿名函数


,data1,data2
key1,,
a,1.837335,0.403841
b,2.429595,2.369401


### **面向列的多函数应用——把自定义函数里传入多个函数名**

In [21]:
tips = pd.read_csv('examples/tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [22]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [23]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

### 定义多个函数

In [24]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/var/folders/g3/9nr_2jgn5gg0yglwn47g6sfm0000gn/T/ipykernel_8909/576261660.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

传入元组数据

In [25]:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/var/folders/g3/9nr_2jgn5gg0yglwn47g6sfm0000gn/T/ipykernel_8909/39108480.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [26]:
grouped.agg({'tip' : np.max, 'size' : 'sum'}) # 用字典映射

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [27]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
           'size' : 'sum'}) # 用嵌套字典映射

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### **以“没有行索引”的形式返回聚合数据**

In [28]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## **groupby的apply方法：一般性的“拆分－应用－合并”**   
> groupby().agg()方法是针对列进行操作的，具有局限性，而apply对整个dataframe使用

In [30]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [29]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [31]:
tips.groupby('smoker').apply(top) # group by的apply是最普适的方法

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [32]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') # 可以传入参数

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### **禁止分组键**

In [33]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### **分位数和桶分析**

In [34]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})     
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (1.443, 3.217]
1    (-2.105, -0.331]
2     (-0.331, 1.443]
3     (-0.331, 1.443]
4    (-2.105, -0.331]
5     (-0.331, 1.443]
6    (-2.105, -0.331]
7     (-0.331, 1.443]
8    (-2.105, -0.331]
9     (-0.331, 1.443]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.886, -2.105] < (-2.105, -0.331] < (-0.331, 1.443] < (1.443, 3.217]]

In [ ]:
def get_stats(group):
            return {'min': group.min(), 'max': group.max(),
             'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()


### **示例：用特定于分组的值填充缺失值**

### **示例：随机采样和排列**

### **示例：分组加权平均数和相关系数**

### **示例：组级别的线性回归**

## **透视表和交叉表**

### **交叉表：crosstab**